Projet réalisé par :  
Mahmoud Benboubker  
Nicolas Calligaro  
Aïcha Lalhou  


# Notebook 0 : Soumission des résultats

Ce notebook permet de produire le fichier csv solution

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Import des librairies de Nicolas 
from IpyTools import *
from IotTools import *
pd.options.mode.chained_assignment = None  # default='warn'


# Import des modèles à utiliser
from sklearn.ensemble import VotingRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import ExtraTreeRegressor

### Import des données

In [3]:
path=""

In [4]:
df_mess_train = pd.read_csv(path+'mess_train_list.csv')
df_mess_test = pd.read_csv(path+'mess_test_list.csv')
pos_train = pd.read_csv(path+'pos_train_list.csv')


### Préprocessing des données d'entrainement

In [5]:
X_train = df_mess_train
listOfBs = np.union1d(df_mess_train.bsid.unique(),df_mess_test.bsid.unique())

In [6]:
X_train=Correct_Bases(df_mess_train)

Nous avons 27 bases outliers
Base 9949 non vu
il reste 0 base avec lat >60


In [7]:
X_train = X_train[~X_train.did.isin([476212., 476830., 476861., 476256,477201, 476829.,476609.,476327,476315,476835,476598,476889,474192,473288])]
X_train.shape

(24967, 8)

#### Création de la matrice des features

In [8]:
df_feat, id_list=feat_mat_const(X_train, listOfBs)

y_full = ground_truth_const(X_train, pos_train, id_list)
y_full.shape,df_feat.shape

((4122, 3), (4122, 273))

#### Implémentation du modèle de prediction
On utilise un modèle VotingRegressor qui est l'aggregation des 5 modèles suivant :

In [9]:
def get_tuned_estimators(coord) : 
    return [('RandomForestRegressor_lat', RandomForestRegressor(**get_hyperparameter("RandomForestRegressor",coord))),
      ('ExtrasTreeRegressor', ExtraTreeRegressor(**get_hyperparameter("ExtraTreeRegressor",coord))),
     ('GradientBoostingRegressor', GradientBoostingRegressor(**get_hyperparameter("GradientBoostingRegressor",coord))),
     ('XGBRegressor', xgb.XGBRegressor(**get_hyperparameter("XGBRegressor",coord))),
     ('BaggingRegressor', BaggingRegressor(**get_hyperparameter("BaggingRegressor",coord)))]

In [10]:
estimators_lat = get_tuned_estimators("lat")
estimators_lng = get_tuned_estimators("lng")

#### Entrainement du VotingRegressor

In [11]:
reg_lat = VotingRegressor(estimators=estimators_lat,n_jobs=-1)
reg_long = VotingRegressor(estimators=estimators_lng, n_jobs=-1)

In [12]:
reg_lat.fit(df_feat,y_full.lat)
reg_long.fit(df_feat,y_full.lng)

VotingRegressor(estimators=[('RandomForestRegressor_lat',
                             RandomForestRegressor(criterion='mae',
                                                   max_depth=10,
                                                   n_estimators=25)),
                            ('ExtrasTreeRegressor',
                             ExtraTreeRegressor(criterion='friedman_mse',
                                                max_depth=8, splitter='best')),
                            ('GradientBoostingRegressor',
                             GradientBoostingRegressor(learning_rate=0.2,
                                                       max_depth=4,
                                                       subsample=0.8)),
                            ('XGBRegressor',
                             XGBRegress...
                                          max_delta_step=None, max_depth=4,
                                          min_child_weight=None, missing=nan,
                   

### Préprocessing des données de test

In [13]:
X_test = df_mess_test
X_test=Correct_Bases(df_mess_test)
df_feat_test, id_list_test=feat_mat_const(X_test, listOfBs)

Nous avons 23 bases outliers
Correction manuelle de la bsid 9949
il reste 0 base avec lat >60


#### Prédiction des données de test

In [14]:
preds_lat = reg_lat.predict(df_feat_test)
preds_lng = reg_long.predict(df_feat_test)

### Création du fichier final
On crée un nouveau DataFrame comprenant la longitude et latitude prédites pour chaque message.

In [15]:
df_feat_test["lat"] = preds_lat
df_feat_test["lng"] = preds_lng
df_feat_test["messid"] = df_feat_test.index
df_feat_test.head()

,879,911,921,944,980,1012,1086,1092,1120,1131,...,min_x,max_x,mean_y,min_y,max_y,BCW_lat,BCW_lng,lat,lng,messid
messid,,,,,,,,,,,,,,,,,,,,,
573be2503e952e191262c351,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,39.728651,39.728651,-105.163032,-105.163032,-105.163032,39.728651,-105.163032,39.708135,-105.065984,573be2503e952e191262c351
573c05f83e952e1912758013,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,39.728651,39.783211,-105.094321,-105.165355,-105.053676,39.782587,-105.088273,39.775528,-105.081089,573c05f83e952e1912758013
573c0796f0fe6e735a66deb3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,39.612745,39.704887,-105.028319,-105.044371,-105.008827,39.660943,-105.040357,39.687718,-105.010542,573c0796f0fe6e735a66deb3
573c08d2864fce1a9a0563bc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,39.782113,39.782113,-105.072701,-105.072701,-105.072701,39.782113,-105.072701,39.781838,-105.073687,573c08d2864fce1a9a0563bc
573c08ff864fce1a9a0579b0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,39.612745,39.704887,-105.028319,-105.044371,-105.008827,39.660387,-105.039809,39.687882,-105.007940,573c08ff864fce1a9a0579b0


On réalise une jointure entre le Dataframe créé et du dataframe *df_mess_test*

In [19]:
df_mess_test_final = df_mess_test.join(other = df_feat_test[["messid","lat","lng"]],on="messid", how = 'left', lsuffix = '_left', rsuffix = '_right')
df_mess_test_final["messid"] = df_mess_test_final["messid_left"]

On crée le fichier résultat

In [20]:
pos_test = df_mess_test_final[["lat","lng","messid"]]
pos_test.to_csv("pred_pos_test_list.csv",index=False)

Aperçu du ficher produit

In [21]:
pd.read_csv("pred_pos_test_list.csv")

,lat,lng,messid
0,39.708135,-105.065984,573be2503e952e191262c351
1,39.775528,-105.081089,573c05f83e952e1912758013
2,39.775528,-105.081089,573c05f83e952e1912758013
3,39.775528,-105.081089,573c05f83e952e1912758013
4,39.775528,-105.081089,573c05f83e952e1912758013
...,...,...,...
29281,39.672754,-105.015520,58485da512f14360d78d5378
29282,39.672754,-105.015520,58485da512f14360d78d5378
29283,39.674540,-105.017256,58486d71e541cd0e1337e073
29284,39.674540,-105.017256,58486d71e541cd0e1337e073
